In [1]:
#First we will Import required libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import requests # library to handle requests
from bs4 import BeautifulSoup
import requests
print('Libraries imported.')

Libraries imported.


Now we prepare our web scraping code using BeautifulSoup


In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).content, 
content = BeautifulSoup(requests.get(url).content, 'lxml')
table = content.find('table') 
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []
# create a list with the data scraped from url 
for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())

Now store the data into a dataframe called df_toronto_codes

In [4]:
df_toronto_codes = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_toronto_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto_codes.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


 Now we need to process cells that have assigned borough.

In [5]:
 # Ignore cells with a borough that is Not assigned.
df_toronto_codes['Borough'].replace('Not assigned', np.nan, inplace=True)
df_toronto_codes.dropna(subset=['Borough'], inplace=True)


Now if a cell has a borough but a Not assigned neighborhood, then we assign neighborhood same as the boroug


In [6]:
df_toronto_codes['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

Combining neighborhoods in each postal code area, to one row with the neighborhoods separated with a comma 

In [7]:
df_toronto_codes = df_toronto_codes.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto_codes.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto_codes.head(8)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


Finally here is the neighborhood data available for city of toronto

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_codes['Borough'].unique()),
        df_toronto_codes.shape[0]
    )
)


The dataframe has 11 boroughs and 103 neighborhoods.
